## Wrangling Files and Data

Reading files with unstructured data. 

Import pandas as pd:

In [87]:
import pandas as pd

From the Berkeley Earth surface temperature dataset, download localized data for New York City:

In [88]:
! curl -o ./data/nyc_temp.txt http://berkeleyearth.lbl.gov/auto/Local/TAVG/Text/40.99N-74.56W-TAVG-Trend.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  313k  100  313k    0     0  1495k      0 --:--:-- --:--:-- --:--:-- 1492k


Inspect the file:

`! head ./data/nyc_temp.txt`

This file is not well-formatted. Use head/tail to narrow the selection:

`! head -72 nyc_temp.txt | tail -8`

In [89]:
! head -72 nyc_temp.txt | tail -8

% 
%                  Monthly          Annual          Five-year        Ten-year        Twenty-year
% Year, Month,  Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.
 
  1743    11    -2.276  1.665       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1743    12       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1744     1       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN
  1744     2       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN       NaN    NaN


### Reading and Formatting Files

Create a list of column names:

`col_names = ['year', 'month', 'monthly_anom']`

Supply the following arguments in `read_csv`:

`header=None`

`usecols=[0,1,2]`

`names = col_names`

`delim_whitespace=True`

`comment="%"`

`temp = pd.read_csv('nyc_temp.txt', header=None, delim_whitespace=True, usecols=[0,1,2], names=col_names, comment="%")`



In [96]:
col_names = ['year', 'month', 'monthly_anom']
temp = pd.read_csv('nyc_temp.txt', header=None, delim_whitespace=True, usecols=[0,1,2], names=col_names, comment="%")
temp

year  month  monthly_anom
0     1743     11        -2.276
1     1743     12           NaN
2     1744      1           NaN
3     1744      2           NaN
4     1744      3           NaN
...    ...    ...           ...
3234  2013      5         0.574
3235  2013      6         0.982
3236  2013      7         2.172
3237  2013      8        -0.659
3238  2013      9        -0.172

[3239 rows x 3 columns]

### Create a DateTime Index

`date_df = temp.drop('monthly_anom', axis=1)`

`date_df["day"] = 1`

`date_index = pd.DatetimeIndex(pd.to_datetime(date_df))`

`temp = temp.set_index(date_index).drop(["year", "month"], axis=1)`

In [140]:
col_names = ['year', 'month', 'monthly_anom']
temp = pd.read_csv('nyc_temp.txt', header=None, usecols=[0, 1, 2], names=col_names, delim_whitespace=True, comment='%')

date_df = temp.drop('monthly_anom', axis=1)
date_df["day"] = 1
date_index = pd.DatetimeIndex(pd.to_datetime(date_df))
temp = temp.set_index(date_index).drop(["year", "month"], axis=1)
temp

monthly_anom
1743-11-01        -2.276
1743-12-01           NaN
1744-01-01           NaN
1744-02-01           NaN
1744-03-01           NaN
...                  ...
2013-05-01         0.574
2013-06-01         0.982
2013-07-01         2.172
2013-08-01        -0.659
2013-09-01        -0.172

[3239 rows x 1 columns]

### Fill Missing Values

`temp = temp.fillna(method="ffill")`

In [142]:
temp = temp.fillna(method="ffill")
temp

monthly_anom
1743-11-01        -2.276
1743-12-01        -2.276
1744-01-01        -2.276
1744-02-01        -2.276
1744-03-01        -2.276
...                  ...
2013-05-01         0.574
2013-06-01         0.982
2013-07-01         2.172
2013-08-01        -0.659
2013-09-01        -0.172

[3239 rows x 1 columns]

## Working with Large Datasets

Using the `chunksize` argument of `read_csv` to parse or perfrom operations on large files

In [93]:
filename = "311_Cases2019.csv"
c_size = 200000

for sf_chunk in pd.read_csv(filename, chunksize=c_size):
    print (sf_chunk.shape)

(200000, 47)
(200000, 47)
(200000, 47)
(18764, 47)


In [94]:
for sf_chunk in pd.read_csv(filename, chunksize=c_size):
    sf_chunk = sf_chunk.iloc[:, 0:18]
    sf_chunk = sf_chunk.fillna("None")
    date_index = pd.DatetimeIndex(pd.to_datetime(sf_chunk["Opened"]))
    sf_chunk.loc[:,"Opened"] = date_index
    sf_chunk.loc[:,"Weekday"] = date_index.weekday_name
    print (sf_chunk["Neighborhood"])

0            Hayes Valley
1            Potrero Hill
2         Diamond Heights
3                 Mission
4               Japantown
               ...       
199995       Outer Sunset
199996    South of Market
199997          Oceanview
199998      Hunters Point
199999           Nob Hill
Name: Neighborhood, Length: 200000, dtype: object
200000    Mission Dolores
200001    Dolores Heights
200002            Mission
200003            Mission
200004            Mission
               ...       
399995    South of Market
399996            Mission
399997            Mission
399998           Parkside
399999            Mission
Name: Neighborhood, Length: 200000, dtype: object
400000            Tenderloin
400001        Haight Ashbury
400002       Dolores Heights
400003                  None
400004            Tenderloin
                 ...        
599995       Dolores Heights
599996    Financial District
599997                Marina
599998          Outer Sunset
599999       South of Market
Name: Nei

In [143]:
from collections import defaultdict

categories = defaultdict(int)

#for sf_chunk in pd.read_csv(filename, chunksize=c_size):
#   for c in sf_chunk["Category"]:
#        categories[c] += 1
#categories

### NaN Functions

`import numpy as np`

`df = pd.DataFrame(np.random.randn(3, 3), index=['a', 'c', 'e'], columns=['one','two', 'three'])`

`print ("Original" + "\n")`

`print (df)`

`df = df.reindex(['a', 'b', 'c'])`

`print ("\n"  + "Reindexed:" + '\n')`

`print (df)`

`print ("\n"  + "NaN replaced with '0':" + "\n")`

`print (df.fillna(0))`

In [137]:
import numpy as np

df = pd.DataFrame(np.random.randn(3, 3), index=['a', 'c', 'e'], columns=['one','two', 'three'])
print ("Original" + "\n")
print (df)
df = df.reindex(['a', 'b', 'c'])
print ("\n"  + "Reindexed:" + '\n')
print (df)
print ("\n"  + "NaN replaced with '0':" + "\n")
print (df.fillna(0))

Original

        one       two     three
a  1.982738  0.486554  1.536442
c -0.854264  0.861779  0.962401
e -1.045314 -0.060082 -1.786305

Reindexed:

        one       two     three
a  1.982738  0.486554  1.536442
b       NaN       NaN       NaN
c -0.854264  0.861779  0.962401

NaN replaced with '0':

        one       two     three
a  1.982738  0.486554  1.536442
b  0.000000  0.000000  0.000000
c -0.854264  0.861779  0.962401


In [138]:
df = pd.DataFrame(np.random.randn(3, 3), index=['a', 'c', 'e'], columns=['one','two', 'three'])
print ("Original" + "\n")
print (df)
df = df.reindex(['a', 'b', 'c'])
print ("\n"  + "Reindexed:" + '\n')
print (df)
df = df.dropna()
print ("\n"  + "After Dropping NaN rows:" + '\n')
print (df)

Original

        one       two     three
a  1.064932  2.136276 -0.430257
c -1.537479 -1.293200  0.211070
e -1.499677 -0.138826 -1.030252

Reindexed:

        one       two     three
a  1.064932  2.136276 -0.430257
b       NaN       NaN       NaN
c -1.537479 -1.293200  0.211070

After Dropping NaN rows:

        one       two     three
a  1.064932  2.136276 -0.430257
c -1.537479 -1.293200  0.211070
